In [1]:
import pandas as pd
import spacy
from sklearn import model_selection
import numpy as np


from tensorflow import keras
from tensorflow.keras import layers, models

In [2]:
df = pd.read_json(open("../data/data.json", "r", encoding="utf8"))


In [3]:
texts_len = df['text'].apply(len)
df.drop(df[texts_len<50].index, inplace=True)

In [4]:
max_features = 10000  # maximum number of words in vocabulari 5000
max_len = 150  # max length of string

In [5]:
joined_text = df['title'] + df['text']
X = keras.preprocessing.sequence.pad_sequences(list(joined_text), maxlen=max_len, padding='post')
Y = np.array(df['themes'].to_list())

In [6]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.1, random_state=42)

### Model 1 simple Dense Layers

In [7]:
embedding_dim =64
model1 = keras.models.Sequential([
  keras.layers.Embedding(input_dim=max_features,
                           output_dim=embedding_dim,
                           input_length=max_len),
  keras.layers.Flatten(),
  keras.layers.Dense(2000,activation='relu'),
  keras.layers.Dense(500,activation='relu'),
  keras.layers.Dense(100,activation='relu'),
  keras.layers.Dense(len(Y[0]), activation='sigmoid')
])

model1.compile(optimizer='nadam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 64)           640000    
_________________________________________________________________
flatten (Flatten)            (None, 9600)              0         
_________________________________________________________________
dense (Dense)                (None, 2000)              19202000  
_________________________________________________________________
dense_1 (Dense)              (None, 500)               1000500   
_________________________________________________________________
dense_2 (Dense)              (None, 100)               50100     
_________________________________________________________________
dense_3 (Dense)              (None, 157)               15857     
Total params: 20,908,457
Trainable params: 20,908,457
Non-trainable params: 0
____________________________________________

In [8]:
epochs = 5
model1.fit(np.array(X_train), np.array(Y_train),
          #batch_size=128,
          validation_data=(np.array(X_test),np.array(Y_test)),
          epochs=epochs)

Epoch 1/5
373/373 [==============================] - 86s 231ms/step - loss: 20.9122 - accuracy: 0.0062 - val_loss: 20.7860 - val_accuracy: 0.0060
Epoch 2/5
373/373 [==============================] - 87s 232ms/step - loss: 19.6421 - accuracy: 0.0039 - val_loss: 20.6585 - val_accuracy: 0.0038
Epoch 3/5
373/373 [==============================] - 88s 236ms/step - loss: 16.9628 - accuracy: 0.0047 - val_loss: 22.0940 - val_accuracy: 0.0038
Epoch 4/5
373/373 [==============================] - 82s 221ms/step - loss: 13.8873 - accuracy: 0.0116 - val_loss: 25.7754 - val_accuracy: 0.0566
Epoch 5/5
373/373 [==============================] - 83s 223ms/step - loss: 11.7530 - accuracy: 0.3132 - val_loss: 29.1777 - val_accuracy: 0.0581


In [9]:
score = model1.evaluate(np.array(X_test), np.array(Y_test)) 

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

42/42 [==============================] - 1s 14ms/step - loss: 29.1777 - accuracy: 0.0581
Test Score: 29.177698135375977
Test Accuracy: 0.058113206177949905


### Model 2 LSTM

In [10]:
embedding_dim =100 

model2 = keras.Sequential([
    layers.Embedding(max_features, embedding_dim, input_length=max_len),
    layers.SpatialDropout1D(0.2),
    layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    layers.Dense(len(Y[0]), activation='sigmoid')
])

In [11]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 100)          1000000   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 150, 100)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense_4 (Dense)              (None, 157)               15857     
Total params: 1,096,257
Trainable params: 1,096,257
Non-trainable params: 0
_________________________________________________________________


In [13]:
max_features = 10000  # maximum number of words in vocabulari 5000
max_len = 150  # max length of string
batch_size = 128

early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)
history = model2.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[early_stopping_callback])

Epoch 1/5
84/84 [==============================] - 17s 204ms/step - loss: 21.5019 - accuracy: 0.0569 - val_loss: 21.0847 - val_accuracy: 0.0814
Epoch 2/5
84/84 [==============================] - 17s 197ms/step - loss: 20.6137 - accuracy: 0.0705 - val_loss: 20.9907 - val_accuracy: 0.0814
Epoch 3/5
84/84 [==============================] - 17s 199ms/step - loss: 20.5725 - accuracy: 0.0705 - val_loss: 20.9866 - val_accuracy: 0.0814
Epoch 4/5
84/84 [==============================] - 16s 195ms/step - loss: 20.5738 - accuracy: 0.0705 - val_loss: 20.9616 - val_accuracy: 0.0814
Epoch 5/5
84/84 [==============================] - 17s 201ms/step - loss: 20.5550 - accuracy: 0.0705 - val_loss: 20.9455 - val_accuracy: 0.0814


In [16]:
accr = model2.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

42/42 [==============================] - 1s 15ms/step - loss: 20.7533 - accuracy: 0.0732
Test set
  Loss: 20.753
  Accuracy: 0.073


In [17]:
output_dim = 100

inputs = keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, output_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

predictions = layers.Dense(len(Y[0]), activation='softmax', name="predictions")(x)

model3 = keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [18]:
model3.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 100)         1000000   
_________________________________________________________________
dropout (Dropout)            (None, None, 100)         0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 128)         89728     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         114816    
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16512 

In [19]:
model3.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

Epoch 1/5
84/84 [==============================] - 4s 49ms/step - loss: 0.1435 - accuracy: 0.0490 - val_loss: 0.1406 - val_accuracy: 0.0814
Epoch 2/5
84/84 [==============================] - 4s 46ms/step - loss: 0.1377 - accuracy: 0.0585 - val_loss: 0.1400 - val_accuracy: 0.0814
Epoch 3/5
84/84 [==============================] - 4s 47ms/step - loss: 0.1367 - accuracy: 0.0646 - val_loss: 0.1399 - val_accuracy: 0.0814
Epoch 4/5
84/84 [==============================] - 4s 47ms/step - loss: 0.1357 - accuracy: 0.0662 - val_loss: 0.1401 - val_accuracy: 0.0814
Epoch 5/5
84/84 [==============================] - 4s 47ms/step - loss: 0.1346 - accuracy: 0.0666 - val_loss: 0.1399 - val_accuracy: 0.0579


In [20]:
accr = model2.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

42/42 [==============================] - 1s 14ms/step - loss: 20.7533 - accuracy: 0.0732
Test set
  Loss: 20.753
  Accuracy: 0.073
